In [1]:
# Importing libraries and Setting Dislplay Options

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats


from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score


#Setting Options
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 100)

D:\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Scraping Fifa2020 Player's Detail Characteristics Data from web site

player_list = []

i = 0
while i < 7000:
    url_list = "https://sofifa.com/?&showCol%5B%5D=pi&showCol%5B%5D=ae&showCol%5B%5D=hi&showCol%5B%5D=wi&showCol%5B%5D=pf&showCol%5B%5D=oa&showCol%5B%5D=pt&showCol%5B%5D=bo&showCol%5B%5D=bp&showCol%5B%5D=gu&showCol%5B%5D=vl&showCol%5B%5D=wg&showCol%5B%5D=rc&showCol%5B%5D=ta&showCol%5B%5D=cr&showCol%5B%5D=fi&showCol%5B%5D=he&showCol%5B%5D=sh&showCol%5B%5D=vo&showCol%5B%5D=ts&showCol%5B%5D=dr&showCol%5B%5D=cu&showCol%5B%5D=fr&showCol%5B%5D=lo&showCol%5B%5D=bl&showCol%5B%5D=to&showCol%5B%5D=ac&showCol%5B%5D=sp&showCol%5B%5D=ag&showCol%5B%5D=re&showCol%5B%5D=ba&showCol%5B%5D=tp&showCol%5B%5D=so&showCol%5B%5D=ju&showCol%5B%5D=st&showCol%5B%5D=sr&showCol%5B%5D=ln&showCol%5B%5D=te&showCol%5B%5D=ar&showCol%5B%5D=in&showCol%5B%5D=po&showCol%5B%5D=vi&showCol%5B%5D=pe&showCol%5B%5D=cm&showCol%5B%5D=td&showCol%5B%5D=ma&showCol%5B%5D=sa&showCol%5B%5D=sl&showCol%5B%5D=tg&showCol%5B%5D=gd&showCol%5B%5D=gh&showCol%5B%5D=gc&showCol%5B%5D=gp&showCol%5B%5D=gr&showCol%5B%5D=tt&showCol%5B%5D=bs&showCol%5B%5D=ir&showCol%5B%5D=pac&showCol%5B%5D=sho&showCol%5B%5D=pas&showCol%5B%5D=dri&showCol%5B%5D=def&showCol%5B%5D=phy&offset="+str(i)
    #print(url_list)
    response = requests.get(url_list)
    #print(response)
    page = response.text
    #print(page)
    soup = BeautifulSoup(page,"lxml")
    #print(soup.prettify)
    rows = soup.find_all("tr")
    #print(rows)
    for row in rows:
        cells = row.find_all('td')
        #print(cells)
        cells_to_string = str(cells)
        cells_sub = (re.sub(re.compile('<.*?>'),'',cells_to_string))
        #print(cells_sub)
        player_list.append(cells_sub)
        result = pd.DataFrame(player_list)
    i += 60

In [3]:
print("Number of All Rows")
print(len(result));
print("")
print("Number of Rows Without Duplicated")
result.drop_duplicates(subset=None, keep="first", inplace=True)
print(len(result))

Number of All Rows
7037

Number of Rows Without Duplicated
6795


In [4]:
#Saving result data frame as a Pickle File
result.to_pickle("PlayerListInitialPickle")

In [5]:
#Reading result from Pickle File as a data frame
result = pd.read_pickle("PlayerListInitialPickle")

In [6]:
# Getting table name (header)
table_players = soup.find("table",attrs={"class":"table table-hover persist-area"})
header_players = table_players.find_all("th")
column_name_players = []
for row in header_players:
    column_name_players.append(row.text)
column_name_players

['',
 'Name',
 'Age',
 'OVA',
 'POT',
 'Team & Contract',
 'ID',
 'Height',
 'Weight',
 'foot',
 'BOV',
 'BP',
 'Growth',
 'Value',
 'Wage',
 'Release Clause',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading Accuracy',
 'Short Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK Accuracy',
 'Long Passing',
 'Ball Control',
 'Movement',
 'Acceleration',
 'Sprint Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'Goalkeeping',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes',
 'Total Stats',
 'Base Stats',
 'IR',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY',
 'Hits']

In [7]:
# Setting columns names
new_column_name_players = [
 'Name',
 'Age',
 'Overall',
 'Potential',
 'Team',
 'Height',
 'Weight',
 'Foot',
 'Best_Overall',
 'Position',
 'Growth',
 'Value',
 'Wage',
 'Release_Clause',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading_Accuracy',
 'Short_Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK_Accuracy',
 'Long_Passing',
 'Ball_Control',
 'Movement',
 'Acceleration',
 'Sprint_Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot_Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long_Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing_Tackle',
 'Sliding_Tackle',
 'Goalkeeping',
 'GK_Diving',
 'GK_Handling',
 'GK_Kicking',
 'GK_Positioning',
 'GK_Reflexes',
 'Total_Stats',
 'Base_Stats',
 'Int_Reputation',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY']

In [8]:
result

,0
0,[]
1,"[\n\n\n\n\nFIFA 21 Nov 6, 202079 ↗ 82 Shot Pow..."
2,"[\n, \n I. SarrRM ST, 22, 78, 87, \n\n\n\nWatf..."
3,"[\n, \n CristianoLM ST RM, 33, 72, 72, \n\n\n\..."
4,"[\n\n\n\n\nFIFA 21 Nov 6, 202066 ↘ 65 Potentia..."
5,"[\n, \n V. GrifoLM CF, 27, 78, 78, \n\n\n\nSC ..."
6,"[\n, \n Cristiano RonaldoST LW, 35, 92, 92, \n..."
7,"[\n\n\n\n\nFIFA 21 Nov 6, 202074 ↗ 75 Short Pa..."
8,"[\n, \n L. WaldschmidtCF ST, 24, 76, 83, \n\n\..."
9,"[\n, \n 20 O. ValakariCM CAM, 19, 64, 77, \n\n..."


In [9]:
#Data Cleaning
result = pd.read_pickle("PlayerListInitialPickle")

result = result[0].str.split(',',expand=True)


result[1] = result[1].str.replace("\n ","")

chars_to_remove = ["GK","RB","RCB","CB","LCB","LB","RWB","RDM","CDM","LDM","LWB","RM","RCM","CM","LCM","LM","LAM","CAM","RAM","RW","RF","CF","LF","LW","LS","ST","RS"]
for char in chars_to_remove:
    result[1]=result[1].str.replace(char,"")
    result[1]=result[1].str.lstrip()

result[5] = result[5].str.replace("\n\n\n\n","")
result[5] = result[5].str.split("\n").str[0]
result[5] = result[5].str.strip()

result[6] = result[6].replace(r'Loan', np.nan, regex=True)
result = result.iloc[1:]

result_WoLoan = result[result[6].notna()]

result_WoLoan.drop(result_WoLoan.columns[[0,6,66,67,68,69]],axis=1, inplace=True)

mask_loan = result[6].isnull()

result_loan = result.loc[mask_loan].shift(-1, axis=1)

result_loan.drop_duplicates(subset=[6], keep="first", inplace=True)

result_loan.drop(result_loan.columns[[5,6,66,67,68,69]],axis=1, inplace=True)

result_WoLoan.columns = new_column_name_players

result_loan.columns = new_column_name_players

result_master = pd.concat([result_WoLoan,result_loan]).reset_index()
del result_master["index"]

result_master.drop_duplicates(subset=None, keep="first",inplace=True)

#result_master

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
#Cleaning some characters

result_master = result_master[~result_master["Height"].str.contains("~")]
result_master = result_master[~result_master["Age"].str.contains("\n")]
result_master = result_master[result_master["Height"].str.contains("'")]

In [11]:
#Converting Height column to cm

result_master["Height"] = result_master["Height"].str[:-1]

def parse_height(height):
    # format: 5'11"
    height = height.split("'")
    feet = float(height[0])
    inch = float(height[1])
    return ((feet*30.48) + (inch*2.54))

result_master["Height"]=round(result_master["Height"].apply(lambda x:parse_height(x)),0)

In [12]:
#Converting Weight column to kg
result_master["Weight"] = round(result_master["Weight"].str.split("lbs").str[0].astype(float)*0.45359237,0)

In [13]:
#Cleaning to Value column and convert to decimal
result_master["Value"] = result_master["Value"].str.replace("€","")
result_master["Value"] = result_master["Value"].str.replace("M","")
result_master.loc[result_master["Value"].str.contains("K"),"Value"]=result_master["Value"].str.split("K").str[0].astype(float)/1000
result_master["Value"] = round(result_master["Value"].astype(float),2)
result_master = result_master.loc[result_master["Value"]>0]

In [14]:
#Cleaning Wage column and convert to decimal
result_master["Wage"] = result_master["Wage"].str.replace("€","")
result_master.loc[result_master["Wage"].str.contains("K"),"Wage"]=result_master["Wage"].str.split("K").str[0].astype(float)*1000
result_master["Wage"] = round(result_master["Wage"].astype(float)/1000000,5)


In [15]:
#Cleaning Release_Clause column and convert to decimal
result_master["Release_Clause"] = result_master["Release_Clause"].str.replace("€","")
result_master["Release_Clause"] = result_master["Release_Clause"].str.replace("M","")
result_master.loc[result_master["Release_Clause"].str.contains("K"),"Release_Clause"]=result_master["Release_Clause"].str.split("K").str[0].astype(float)/1000
result_master["Release_Clause"] = round(result_master["Release_Clause"].astype(float),2)

In [16]:
#Checking Height,Weight,Value,Wage and Release_Clause columns
result_master.head(10)

,Name,Age,Overall,Potential,Team,Height,Weight,Foot,Best_Overall,Position,Growth,Value,Wage,Release_Clause,Attacking,Crossing,Finishing,Heading_Accuracy,Short_Passing,Volleys,Skill,Dribbling,Curve,FK_Accuracy,Long_Passing,Ball_Control,Movement,Acceleration,Sprint_Speed,Agility,Reactions,Balance,Power,Shot_Power,Jumping,Stamina,Strength,Long_Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing_Tackle,Sliding_Tackle,Goalkeeping,GK_Diving,GK_Handling,GK_Kicking,GK_Positioning,GK_Reflexes,Total_Stats,Base_Stats,Int_Reputation,PAC,SHO,PAS,DRI,DEF,PHY
1,I. Sarr,22,78,87,Watford,185.0,76.0,Right,79,RM,9,31.50,0.01900,63.0,332,68,75,44,69,76,354,83,78,55,59,79,417,95,94,78,76,74,355,84,48,73,70,80,275,53,17,75,72,58,78,62,25,16,21,49,14,13,7,6,9,1844,408,1 ★,94,77,68,81,22,66
2,Cristiano,33,72,72,Kashiwa Reysol,183.0,83.0,Right,73,ST,0,1.50,0.00900,1.9,347,72,71,68,68,68,353,71,72,74,65,71,338,76,74,72,67,49,386,78,68,75,88,77,338,74,56,73,70,65,58,123,41,46,36,48,13,8,11,8,8,1933,416,1 ★,75,73,69,70,48,81
4,V. Grifo,27,78,78,SC Freiburg,180.0,77.0,Right,78,LM,0,15.00,0.02800,27.0,352,86,70,47,76,73,395,82,85,80,67,81,372,73,68,78,74,79,342,81,52,71,63,75,318,52,45,77,74,70,70,111,50,31,30,52,10,12,8,8,14,1942,405,1 ★,70,74,77,81,41,62
5,Cristiano Ronaldo,35,92,92,Juventus,188.0,83.0,Right,92,ST,0,63.00,0.22000,104.0,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,5 ★,89,93,81,89,35,77
7,L. Waldschmidt,24,76,83,SL Benfica,180.0,74.0,Left,77,CAM,7,15.00,0.01300,33.0,347,63,78,59,75,72,343,75,73,63,58,74,377,77,71,78,74,77,363,81,66,70,64,82,310,44,28,80,75,83,71,70,33,19,18,56,14,10,7,9,16,1866,388,1 ★,74,79,69,75,29,62
8,20 O. Valakari,19,64,77,Tromsø IL,188.0,82.0,Left,65,CM,13,0.82,0.00075,1.5,281,55,62,52,68,44,271,56,47,41,65,62,284,62,62,45,62,53,325,65,53,75,72,60,283,48,50,65,67,53,66,138,45,46,47,44,6,9,14,9,6,1626,356,1 ★,62,61,62,57,47,67
11,T. Alexander-Arnold,21,87,92,Liverpool,180.0,69.0,Right,87,RB,5,114.00,0.11000,219.5,365,93,56,69,85,62,426,77,88,85,89,87,391,77,83,74,83,74,370,80,66,88,62,74,371,74,82,76,84,55,81,241,79,83,79,56,14,15,14,6,7,2220,464,2 ★,80,66,87,80,80,71
12,João Félix,20,82,93,Atlético Madrid,180.0,70.0,Right,84,CAM,11,67.50,0.05600,151.9,383,73,82,69,79,80,382,85,81,64,67,85,410,79,83,85,83,80,381,82,79,75,64,81,340,69,38,84,81,68,82,107,30,42,35,66,13,12,14,13,14,2069,432,1 ★,81,81,76,85,40,69
13,E. Haaland,19,84,92,Borussia Dortmund,193.0,88.0,Left,85,ST,8,92.00,0.05600,174.8,357,46,88,67,74,82,342,75,77,62,49,79,394,80,89,75,83,67,415,93,70,78,91,83,358,83,41,86,68,80,83,113,45,38,30,52,7,14,13,11,7,2031,439,2 ★,85,87,63,76,43,85
14,J. Frimpong,19,71,86,Celtic,175.0,70.0,Right,72,RM,15,4.30,0.01500,10.4,276,69,51,50,68,38,316,76,64,47,54,75,423,92,89,88,65,89,307,51,71,74,64,47,289,68,60,59,63,39,74,194,62,67,65,54,12,7,8,13,14,1859,410,1 ★,90,49,64,77,62,68


In [17]:
#Deleting some rows that composure columns is blank (Ronaldinho, Juninho, Xavi etc)
result_master = result_master.loc[result_master["Composure"]!=" "]
result_master = result_master[result_master["Base_Stats"]!=' ']

In [18]:
#Cleaning to IR column
result_master["Int_Reputation"]=result_master["Int_Reputation"].str[:-1]

In [19]:
#Checking null values
result_master.isnull().any()

Name                False
Age                 False
Overall             False
Potential           False
Team                False
Height              False
Weight              False
Foot                False
Best_Overall        False
Position            False
Growth              False
Value               False
Wage                False
Release_Clause      False
Attacking           False
Crossing            False
Finishing           False
Heading_Accuracy    False
Short_Passing       False
Volleys             False
Skill               False
Dribbling           False
Curve               False
FK_Accuracy         False
Long_Passing        False
Ball_Control        False
Movement            False
Acceleration        False
Sprint_Speed        False
Agility             False
Reactions           False
Balance             False
Power               False
Shot_Power          False
Jumping             False
Stamina             False
Strength            False
Long_Shots          False
Mentality   

In [20]:
#Checking columns dtypes
result_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5292 entries, 1 to 6793
Data columns (total 64 columns):
Name                5292 non-null object
Age                 5292 non-null object
Overall             5292 non-null object
Potential           5292 non-null object
Team                5292 non-null object
Height              5292 non-null float64
Weight              5292 non-null float64
Foot                5292 non-null object
Best_Overall        5292 non-null object
Position            5292 non-null object
Growth              5292 non-null object
Value               5292 non-null float64
Wage                5292 non-null float64
Release_Clause      5292 non-null float64
Attacking           5292 non-null object
Crossing            5292 non-null object
Finishing           5292 non-null object
Heading_Accuracy    5292 non-null object
Short_Passing       5292 non-null object
Volleys             5292 non-null object
Skill               5292 non-null object
Dribbling           5292 no

In [21]:
#Converting to int dtypes

cols_to_convert=['Age',
 'Overall',
 'Potential',
 'Best_Overall',
 'Growth',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading_Accuracy',
 'Short_Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK_Accuracy',
 'Long_Passing',
 'Ball_Control',
 'Movement',
 'Acceleration',
 'Sprint_Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot_Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long_Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing_Tackle',
 'Sliding_Tackle',
 'Goalkeeping',
 'GK_Diving',
 'GK_Handling',
 'GK_Kicking',
 'GK_Positioning',
 'GK_Reflexes',
 'Total_Stats',
 'Base_Stats',
 'Int_Reputation',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY']

for col in cols_to_convert:
    result_master[col] = result_master[col].astype("int")

In [22]:
result_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5292 entries, 1 to 6793
Data columns (total 64 columns):
Name                5292 non-null object
Age                 5292 non-null int32
Overall             5292 non-null int32
Potential           5292 non-null int32
Team                5292 non-null object
Height              5292 non-null float64
Weight              5292 non-null float64
Foot                5292 non-null object
Best_Overall        5292 non-null int32
Position            5292 non-null object
Growth              5292 non-null int32
Value               5292 non-null float64
Wage                5292 non-null float64
Release_Clause      5292 non-null float64
Attacking           5292 non-null int32
Crossing            5292 non-null int32
Finishing           5292 non-null int32
Heading_Accuracy    5292 non-null int32
Short_Passing       5292 non-null int32
Volleys             5292 non-null int32
Skill               5292 non-null int32
Dribbling           5292 non-null int32

In [23]:
#Descriptive Statistics of numeric columns
result_master.describe()

,Age,Overall,Potential,Height,Weight,Best_Overall,Growth,Value,Wage,Release_Clause,Attacking,Crossing,Finishing,Heading_Accuracy,Short_Passing,Volleys,Skill,Dribbling,Curve,FK_Accuracy,Long_Passing,Ball_Control,Movement,Acceleration,Sprint_Speed,Agility,Reactions,Balance,Power,Shot_Power,Jumping,Stamina,Strength,Long_Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing_Tackle,Sliding_Tackle,Goalkeeping,GK_Diving,GK_Handling,GK_Kicking,GK_Positioning,GK_Reflexes,Total_Stats,Base_Stats,Int_Reputation,PAC,SHO,PAS,DRI,DEF,PHY
count,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.00000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.00000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000,5292.000000
mean,24.739040,70.432162,76.455593,181.056500,75.037604,71.487906,6.023432,6.652443,0.019781,11.938940,275.988851,55.179327,51.831822,55.810847,64.501323,48.665533,286.922147,62.485450,53.869803,47.455782,58.192933,64.918178,337.057445,68.216553,68.031935,67.539305,66.75000,66.519652,314.817082,63.636810,66.293462,65.701625,66.003401,53.181784,277.449169,59.572184,49.245465,56.355442,59.61678,52.659297,64.430839,147.885676,49.602797,50.406085,47.876795,76.837113,15.466175,15.343537,15.108466,15.364890,15.554044,1716.957483,379.243197,1.270219,70.712963,58.680461,62.634543,68.285525,52.402305,66.527400
std,4.810148,6.804346,5.114463,7.003686,7.323937,6.523039,6.004992,12.241190,0.030598,23.779802,73.123472,18.213671,19.742197,17.354965,13.642512,18.369555,77.555326,17.993646,18.721038,18.251444,14.672948,15.711546,52.662974,14.221018,13.904469,13.893803,8.97682,13.952758,48.444676,12.900986,12.063864,14.848748,12.534797,19.327354,63.512164,17.262859,21.751201,19.452186,13.56592,15.898116,11.318566,64.215890,20.857642,22.354265,22.240765,84.038823,17.452048,16.654059,16.147471,17.048877,17.794207,257.576795,39.816362,0.604210,10.887502,13.919964,10.010288,9.435690,17.679856,9.836108
min,16.000000,47.000000,53.000000,155.000000,53.000000,47.000000,0.000000,0.060000,0.000500,0.000000,42.000000,7.000000,3.000000,5.000000,11.000000,4.000000,58.000000,5.000000,6.000000,5.000000,11.000000,9.000000,138.000000,13.000000,15.000000,19.000000,35.00000,17.000000,133.000000,12.000000,26.000000,13.000000,28.000000,4.000000,64.000000,11.000000,6.000000,3.000000,11.00000,8.000000,14.000000,23.000000,4.000000,7.000000,6.000000,10.000000,2.000000,2.000000,1.000000,2.000000,2.000000,770.000000,239.000000,1.000000,29.000000,16.000000,27.000000,30.000000,15.000000,31.000000
25%,21.000000,66.000000,73.000000,175.000000,70.000000,67.000000,0.000000,1.300000,0.003000,1.700000,250.000000,45.000000,38.000000,48.000000,60.000000,35.000000,254.000000,58.000000,42.000000,34.000000,50.000000,62.000000,310.000000,62.000000,62.000000,60.000000,61.00000,59.000000,283.000000,55.000000,59.000000,59.000000,58.000000,41.000000,249.000000,48.000000,28.000000,49.000000,51.00000,43.000000,58.000000,89.000000,31.000000,30.000000,26.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1578.000000,352.000000,1.000000,65.000000,50.000000,56.000000,63.000000,36.000000,60.000000
50%,24.000000,71.000000,76.000000,180.000000,75.000000,72.000000,5.000000,2.500000,0.010000,4.100000,289.000000,60.000000,57.000000,59.000000,67.000000,51.000000,300.000000,67.000000,57.000000,47.000000,61.000000,69.000000,346.000000,70.000000,70.000000,70.000000,67.00000,69.000000,321.000000,65.000000,67.0000

In [24]:
result_master.head()

,Name,Age,Overall,Potential,Team,Height,Weight,Foot,Best_Overall,Position,Growth,Value,Wage,Release_Clause,Attacking,Crossing,Finishing,Heading_Accuracy,Short_Passing,Volleys,Skill,Dribbling,Curve,FK_Accuracy,Long_Passing,Ball_Control,Movement,Acceleration,Sprint_Speed,Agility,Reactions,Balance,Power,Shot_Power,Jumping,Stamina,Strength,Long_Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing_Tackle,Sliding_Tackle,Goalkeeping,GK_Diving,GK_Handling,GK_Kicking,GK_Positioning,GK_Reflexes,Total_Stats,Base_Stats,Int_Reputation,PAC,SHO,PAS,DRI,DEF,PHY
1,I. Sarr,22,78,87,Watford,185.0,76.0,Right,79,RM,9,31.5,0.019,63.0,332,68,75,44,69,76,354,83,78,55,59,79,417,95,94,78,76,74,355,84,48,73,70,80,275,53,17,75,72,58,78,62,25,16,21,49,14,13,7,6,9,1844,408,1,94,77,68,81,22,66
2,Cristiano,33,72,72,Kashiwa Reysol,183.0,83.0,Right,73,ST,0,1.5,0.009,1.9,347,72,71,68,68,68,353,71,72,74,65,71,338,76,74,72,67,49,386,78,68,75,88,77,338,74,56,73,70,65,58,123,41,46,36,48,13,8,11,8,8,1933,416,1,75,73,69,70,48,81
4,V. Grifo,27,78,78,SC Freiburg,180.0,77.0,Right,78,LM,0,15.0,0.028,27.0,352,86,70,47,76,73,395,82,85,80,67,81,372,73,68,78,74,79,342,81,52,71,63,75,318,52,45,77,74,70,70,111,50,31,30,52,10,12,8,8,14,1942,405,1,70,74,77,81,41,62
5,Cristiano Ronaldo,35,92,92,Juventus,188.0,83.0,Right,92,ST,0,63.0,0.220,104.0,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,5,89,93,81,89,35,77
7,L. Waldschmidt,24,76,83,SL Benfica,180.0,74.0,Left,77,CAM,7,15.0,0.013,33.0,347,63,78,59,75,72,343,75,73,63,58,74,377,77,71,78,74,77,363,81,66,70,64,82,310,44,28,80,75,83,71,70,33,19,18,56,14,10,7,9,16,1866,388,1,74,79,69,75,29,62


In [25]:
result_master.describe(include='object')

,Name,Team,Foot,Position
count,5292,5292,5292,5292
unique,5222,639,2,15
top,J. Rodríguez,FC Barcelona,Right,CB
freq,3,33,3949,883


In [27]:
#Saving Final Player List as csv
result_master.to_csv("dataFinals.csv")

In [28]:
#Reading Final Player List
df= pd.read_csv("dataFinals.csv")

In [29]:
df.drop(df[df["Team"] == 'Huddersfield Town'].index, inplace = True)

In [30]:
df_new=df[df['Age']<22]

In [31]:
df_new.shape

(1578, 65)

In [32]:
df_new.isna().sum()

Unnamed: 0          0
Name                0
Age                 0
Overall             0
Potential           0
Team                0
Height              0
Weight              0
Foot                0
Best_Overall        0
Position            0
Growth              0
Value               0
Wage                0
Release_Clause      0
Attacking           0
Crossing            0
Finishing           0
Heading_Accuracy    0
Short_Passing       0
Volleys             0
Skill               0
Dribbling           0
Curve               0
FK_Accuracy         0
Long_Passing        0
Ball_Control        0
Movement            0
Acceleration        0
Sprint_Speed        0
Agility             0
Reactions           0
Balance             0
Power               0
Shot_Power          0
Jumping             0
Stamina             0
Strength            0
Long_Shots          0
Mentality           0
Aggression          0
Interceptions       0
Positioning         0
Vision              0
Penalties           0
Composure 

In [33]:
df_new.drop(columns=['Name','Team','Foot','Position'],inplace=True)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_new=df[['Age','Overall','Potential','Best_Overall','Growth','Value','Int_Reputation','Wage']]

In [35]:
df_new.columns

Index(['Age', 'Overall', 'Potential', 'Best_Overall', 'Growth', 'Value',
       'Int_Reputation', 'Wage'],
      dtype='object')

In [36]:
#splitting data
from sklearn.model_selection import train_test_split

In [37]:
X=df_new.drop(columns='Value')
y=df_new['Value']

In [38]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=.80,random_state=42)

D:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [39]:
from sklearn.linear_model import LinearRegression

In [40]:
modelLog=LinearRegression()

In [41]:
modelLog.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [42]:
modelLog.predict(X_test)

array([31.03047411, 11.87884757,  4.63479805, ...,  1.48934217,
        1.17857392,  8.7215415 ])

In [43]:

print(f"score train {modelLog.score(X_train,y_train)}")
print(f"score test {modelLog.score(X_test,y_test)}")

score train 0.7106517174154334
score test 0.7209491805678603


In [44]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [45]:
pred_test=modelLog.predict(X_test)

In [46]:
R2_test=r2_score(y_test,pred_test)
R2_test

0.7209491805678603

In [47]:
MAE_test=mean_absolute_error(y_test,pred_test)
MAE_test

4.009550468418822

In [48]:
MSE_test=mean_squared_error(y_test,pred_test)
MSE_test

49.35335803732091

In [49]:
RMSE_test=np.sqrt(MSE_test)
RMSE_test

7.025194519536161

In [50]:
df.columns

Index(['Unnamed: 0', 'Name', 'Age', 'Overall', 'Potential', 'Team', 'Height',
       'Weight', 'Foot', 'Best_Overall', 'Position', 'Growth', 'Value', 'Wage',
       'Release_Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading_Accuracy', 'Short_Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK_Accuracy', 'Long_Passing', 'Ball_Control', 'Movement',
       'Acceleration', 'Sprint_Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot_Power', 'Jumping', 'Stamina', 'Strength', 'Long_Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing_Tackle',
       'Sliding_Tackle', 'Goalkeeping', 'GK_Diving', 'GK_Handling',
       'GK_Kicking', 'GK_Positioning', 'GK_Reflexes', 'Total_Stats',
       'Base_Stats', 'Int_Reputation', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF',
       'PHY'],
      dtype='object')